In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils.remove_cols as remove_cols
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler

# IMPORTIEREN

In [2]:
df = pd.read_csv("immoscout_cleaned_lat_lon_fixed_v7.csv")

/var/folders/mw/x398f5s545bffd989r6xpm100000gp/T/ipykernel_3028/2143901549.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("immoscout_cleaned_lat_lon_fixed_v7.csv")


# BEREINIGEN / IMPUTIEREN

# TRANSFORMIEREN

In [3]:
# create dictionary with object-type information
types_nrs = dict(enumerate(df["type"].unique()))
types_name = {v: k for k, v in types_nrs.items()}

# change object-type to numeric value
df["type"] = df["type"].map(types_name)

# keep only cols with type int or float
df = df.select_dtypes(include=["int64", "float64"])

# remove specific columns (manually choosen in an excel file which columns to remove)
df_remove = pd.read_excel("fixtures/remove_cols.xlsx")
df_remove = remove_cols.remove_chars_space(df_remove, "feature")
dct = remove_cols.get_dct_from_df(df_remove)
# test if dct has same amount of columns as df
assert len(dct) == len(df.columns)
df = df[[k for k, v in dct.items() if v == 1]]

# test if amount of columns match with expectation
assert len(df.columns) == sum(dct.values())
assert df.values.shape[0] != 0

# print df
df.head()

,ForestDensityL,ForestDensityM,ForestDensityS,NoisePollutionRailwayL,NoisePollutionRailwayM,NoisePollutionRailwayS,NoisePollutionRoadL,NoisePollutionRoadM,NoisePollutionRoadS,PopulationDensityL,...,gde_area_forest_percentage,gde_area_nonproductive_percentage,gde_area_settlement_percentage,gde_foreigners_percentage,gde_pop_per_km2,gde_population,gde_private_apartments,gde_tax,price_cleaned,type
0,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,...,51.449275,4.589372,13.285024,9.255663,376.829268,1545.0,686.0,5.89,1150000.0,0
1,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,...,51.449275,4.589372,13.285024,9.255663,376.829268,1545.0,686.0,5.89,1420000.0,1
2,0.163362,0.095877,0.001911,0.0,0.0,0.0,0.334957,0.381257,0.297575,0.325887,...,32.197891,7.137064,49.310624,21.358623,1704.700162,21036.0,10149.0,6.05,720000.0,0
3,0.511176,0.286451,0.090908,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,...,51.449275,4.589372,13.285024,9.255663,376.829268,1545.0,686.0,5.89,1430000.0,2
4,0.333865,0.279276,0.145835,0.0,0.0,0.0,0.133498,0.132933,0.235917,0.190986,...,49.705635,1.177460,15.979815,15.901990,511.008403,6081.0,2638.0,6.30,995000.0,3


# PREPARE DATA

In [4]:
# drop nan because the model can't handle them
df = df.dropna() # 1016 / 13378 = 7.6% of data is lost
# make Matrix y as target with column type
y = df["type"].values[:, np.newaxis]
# make Matrix X as features with all columns except type
X = df.drop(["type"], axis=1).values
assert type(y) == np.ndarray
assert type(X) == np.ndarray
# standardize X
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

array([[ 2.64194273,  3.32482523,  3.30198419, ..., -0.42010596,
        -1.04271332, -0.64676889],
       [-0.60338065, -0.76531049, -0.50523966, ..., -0.42055213,
        -1.36713496, -0.43886317],
       [-0.22362949, -0.61969113, -0.50150005, ..., -0.2916091 ,
        -1.31137499,  0.19970442],
       ...,
       [ 0.28376289,  0.63366458, -0.06385166, ..., -0.42867242,
        -1.31137499, -0.62449328],
       [-0.95165389, -0.65312147, -0.50523966, ..., -0.25939565,
        -1.12381873,  0.35563371],
       [-1.03847093, -0.76531049, -0.50523966, ...,  1.16067314,
         2.13053587, -0.66904451]])

# FIT, PREDICT, SCORE

In [5]:
# define which models to use
models = [
    SVC(),
    LogisticRegression(),
    KNeighborsClassifier(),
    RandomForestClassifier(),
    BaggingClassifier()
]

In [6]:
model_accuracies = {
    "SVC" : 0, 
    "LogisticRegression" : 0,
    "KNeighborsClassifier" : 0,
    "RandomForestClassifier" : 0,
    "BaggingClassifier" : 0
}

In [7]:
for model in models:
    # get classifier for one of the models e.g. SVC or LogisticRegression etc..
    clf = OneVsRestClassifier(model).fit(X_train.copy(), y_train.copy())
    # predict y
    y_pred = clf.predict(X_test.copy())
    # add accuracy score of y_pred vs y_true to accuracy dictionary
    model_accuracies[model.__class__.__name__] = accuracy_score(y_test, y_pred)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
# print accuracies of all models
model_accuracies

{'SVC': 0.5438738374443995,
 'LogisticRegression': 0.5050545895673272,
 'KNeighborsClassifier': 0.5216336433481601,
 'RandomForestClassifier': 0.5640921957137081,
 'BaggingClassifier': 0.5653052972098666}